# Image Classification
## Deep Learning

## Download The Oxford-IIIT Pet Dataset

In [ ]:
!apt -q install aria2


In [ ]:
!wget https://academictorrents.com/download/b18bbd9ba03d50b0f7f479acc9f4228a408cecc1.torrent -O tiny-imagenet.torrent

In [ ]:
!aria2c --seed-time=0 -d /content -o tiny-imagenet-200.zip tiny-imagenet.torrent

In [ ]:
!unzip -q /content/tiny-imagenet-200.zip -d /content/